In [8]:
!pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 kB 3.7 MB/s eta 0:00:00


In [13]:
!pip install bokeh==2.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 57.8 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.6.2
    Uninstalling bokeh-3.6.2:
      Successfully uninstalled bokeh-3.6.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 1.5.5 requires bokeh<3.7.0,>=3.5.0, but you have bokeh 2.4.3 which is incompatible.
holoviews 1.20.0 requires bokeh>=3.1, but you have bokeh 2.4.3 which is incompatible.


In [2]:
import os
import math
import asyncio
import nest_asyncio

import numpy as np
import osmnx as ox
import pandas as pd
# import pydeck as pdk
import geopandas as gpd

from glob import glob
# from pyppeteer import launch
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options

from bokeh.io import output_file
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors

import plotly.express as px
import plotly.graph_objects as go

In [3]:
def latlon_to_web_mercator(lat, lon):
    """
    Convert latitude and longitude to Web Mercator projection.
    """
    k = 6378137  # Earth's radius in meters
    x = lon * (k * math.pi / 180.0)
    y = math.log(math.tan((90 + lat) * math.pi / 360.0)) * k
    return x, y

# Extract x and y coordinates for polygons, ignoring z-coordinate
def extract_coords(geometry):
    if geometry.geom_type == "Polygon":
        # Extract x, y from 3D coordinates
        coords = [(coord[0], coord[1]) for coord in geometry.exterior.coords]
    elif geometry.geom_type == "MultiPolygon":
        # Handle MultiPolygons by extracting all exterior rings
        coords = [(coord[0], coord[1]) for polygon in geometry for coord in polygon.exterior.coords]
    else:
        coords = []  # Handle unsupported geometries gracefully
    return coords

In [4]:
# Load GeoPandas DataFrame
gdf = gpd.read_file("Centros Comerciales.kml")
gdf.head()

,Name,Description,geometry
0,City Mall,,"POLYGON Z ((-79.91049 -2.14146 0, -79.90921 -2..."
1,Riocentro Norte,,"POLYGON Z ((-79.90559 -2.12627 0, -79.90735 -2..."
2,Mall del Río,,"POLYGON Z ((-79.90537 -2.12544 0, -79.90532 -2..."
3,Mall del Sol,,"POLYGON Z ((-79.89463 -2.1547 0, -79.8946 -2.1..."
4,La Rotonda,,"POLYGON Z ((-79.90882 -2.13974 0, -79.90854 -2..."


In [5]:
# Extract coordinates and convert to Web Mercator
gdf["xs"] = gdf.geometry.apply(
    lambda geom: [latlon_to_web_mercator(lat, lon)[0] for lon, lat in extract_coords(geom)]
)
gdf["ys"] = gdf.geometry.apply(
    lambda geom: [latlon_to_web_mercator(lat, lon)[1] for lon, lat in extract_coords(geom)]
)

In [6]:
# Create a Bokeh ColumnDataSource
source = ColumnDataSource({
    "xs": gdf["xs"],
    "ys": gdf["ys"],
    "name": gdf["Name"],  # Add additional columns if needed
})

source

ColumnDataSource(id='1002', ...)

In [8]:
idx_mall = 0

gdf_mall = gdf.iloc[idx_mall]
gdf_mall

,0
Name,City Mall
Description,
geometry,POLYGON Z ((-79.9104909638874 -2.1414649111780...
xs,"[-8895595.163140435, -8895452.230516383, -8895..."
ys,"[-238442.30472093908, -238508.23093141706, -23..."


In [17]:
# Center of the map
lon, lat = list(gdf_mall.geometry.boundary.centroid.coords)[0]
lon, lat

(-79.90951320936493, -2.1409155511014633)

In [18]:
# Convert lat/lon to Web Mercator coordinates
center_x, center_y = latlon_to_web_mercator(lat, lon)

# Define zoom level
zoom_level = 16  # Higher zoom level = closer view
tile_size = 256  # Tile size for Web Mercator
earth_circumference = 40075016.686  # Earth's circumference in meters
initial_resolution = earth_circumference / tile_size  # Resolution at zoom level 0
resolution = initial_resolution / (2 ** zoom_level)  # Resolution at the desired zoom level

# Calculate x_range and y_range based on zoom level
range_size = resolution * tile_size  # Size of the range in meters
x_range = (center_x - range_size / 2, center_x + range_size / 2)
y_range = (center_y - range_size / 2, center_y + range_size / 2)

# Define the map with centered coordinates and zoom level
p = figure(
    x_range=x_range,
    y_range=y_range,
    x_axis_type="mercator",
    y_axis_type="mercator",
    title=f"Map with GeoPandas Data (Zoom Level {zoom_level})"
)

tile_provider = get_provider(Vendors.ESRI_IMAGERY)  # Using ESRI Satellite Imagery
# tile_provider = get_provider(Vendors.OSM)  # Using ESRI Satellite Imagery
p.add_tile(tile_provider)

# Plot polygons
p.patches(xs="xs", ys="ys", source=source, fill_alpha=0.5, line_width=1, color="blue")

n = 7
output_file(f"map_debug{n}.html")

# Show the map
# show(p)

In [19]:
p

Figure(id='1007', ...)